In [1]:
import sys
sys.path.insert(1, '/Users/rpeck/Source/h2o-3/h2o-py')
import h2o
import time

In [2]:
import pkg_resources
ver = pkg_resources.get_distribution("h2o").version
print "H2O PYTHON PACKAGE VERSION: " + str(ver)

H2O PYTHON PACKAGE VERSION: 3.0.0.99999


In [3]:
# Explore a typical Data Science workflow with H2O and Python
#
# Goal: assist the manager of CitiBike of NYC to load-balance the bicycles
# across the CitiBike network of stations, by predicting the number of bike
# trips taken from the station every day.  Use 10 million rows of historical
# data, and eventually add weather data.


# Connect to a cluster
h2o.init(ip="127.0.0.1", port=54321, strict_version_check=False)

H2O cluster uptime:,35 seconds 77 milliseconds
H2O cluster version:,3.7.0.3300
H2O cluster name:,rpeck
H2O cluster total nodes:,1
H2O cluster total memory:,3.56 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [4]:
# Set this to True if you want to fetch the data directly from S3.
# This is useful if your cluster is running in EC2.
data_source_is_s3 = False

def mylocate(s):
    if data_source_is_s3:
        return "s3n://h2o-public-test-data/" + s
    else:
        return h2o.locate(s)

In [5]:
# Pick either the big or the small demo.
# Big data is 10M rows
small_test = [mylocate("bigdata/laptop/citibike-nyc/2013-10.csv")]
big_test =   [mylocate("bigdata/laptop/citibike-nyc/2013-07.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2013-08.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2013-09.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2013-10.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2013-11.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2013-12.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-01.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-02.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-03.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-04.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-05.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-06.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-07.csv"),
              mylocate("bigdata/laptop/citibike-nyc/2014-08.csv")]

# ----------

# 1- Load data - 1 row per bicycle trip.  Has columns showing the start and end
# station, trip duration and trip start time and day.  The larger dataset
# totals about 10 million rows
print "Import and Parse bike data"
data = h2o.import_frame(path=big_test)

Import and Parse bike data

Parse Progress: [##################################################] 100%


In [6]:
# ----------

# 2- light data munging: group the bike starts per-day, converting the 10M rows
# of trips to about 140,000 station&day combos - predicting the number of trip
# starts per-station-per-day.

# Convert start time to: Day since the Epoch
startime = data["starttime"]
secsPerDay=1000*60*60*24
data["Days"] = (startime/secsPerDay).floor()
data.describe()

Rows:10,407,546 Cols:16

Chunk compression summary:



chunk_type,chunk_name,count,count_percentage,size,size_percentage
C1,1-Byte Integers,53,6.25,9.9 MB,1.6125029
C1N,1-Byte Integers (w/o NAs),53,6.25,9.9 MB,1.6125029
C1S,1-Byte Fractions,106,12.5,19.9 MB,3.2252688
C2,2-Byte Integers,290,34.198112,108.5 MB,17.62338
C4,4-Byte Integers,28,3.3018868,21.2 MB,3.4463155
C4S,4-Byte Fractions,39,4.5990567,30.2 MB,4.8972745
C8,64-bit Integers,106,12.5,158.8 MB,25.792233
C8D,64-bit Reals,173,20.400944,257.3 MB,41.79052



Frame distribution summary:



,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.58:54321,615.7 MB,10407546.0,53.0,848.0
mean,615.7 MB,10407546.0,53.0,848.0
min,615.7 MB,10407546.0,53.0,848.0
max,615.7 MB,10407546.0,53.0,848.0
stddev,0 B,0.0,0.0,0.0
total,615.7 MB,10407546.0,53.0,848.0


tripduration starttime stoptime start station id start station name start station latitude start station longitude end station id end station name end station latitude end station longitude bikeid usertype birth year gender Days
------- -------------- ----------------- ----------------- ------------------ ----------------------- ------------------------ ------------------------- ---------------- --------------------------- ---------------------- ----------------------- ------------- -------------- ------------- -------------- -------------
type int time time int enum real real int enum real real int enum int int int
mins 60.0 1.372662e+12 1.372662242e+12 72.0 0.0 40.680342423 -74.01713445 72.0 0.0 40.680342423 -74.01713445 14529.0 0.0 1899.0 0.0 15887.0
mean 868.968726057 1.39099985823e+12 1.39100072718e+12 444.858835407 NaN 40.7343819823 -73.9910570182 445.259785544 NaN 40.7340868895 -73.9911707799 17895.6618358 0.880131781306 1975.79892394 1.08414654137 16099.0245177
maxs 6250750.0 1.409554787e+12 1.409563605e+12 3002.0 339.0 40.771522 -73.9500479759 3002.0 339.0 40.771522 -73.9500479759 21689.0 1.0 1998.0 2.0 16314.0
sigma 2985.10540532 11806578171.7 11806555707.8 355.755989765 NaN 0.0197100508736 0.0123453320185 360.070380844 NaN 0.0197309578633 0.0124311861598 1938.80517884 0.324807387506 11.132784905 0.563019777794 136.647269305
zeros 0 0 0 0 56836 0 0 0 55167 0 0 0 1247534 0 1248517 0
missing 0 0 0 0 0 0 0 0 0 0 0 0 0 1247644 0 0
0 634.0 1.372662e+12 1.372662634e+12 164.0 E 47 St & 2 Ave 40.75323098 -73.97032517 504.0 1 Ave & E 15 St 40.73221853 -73.98165557 16950.0 Customer nan 0.0 15887.0
1 1547.0 1.372662002e+12 1.372663549e+12 388.0 W 26 St & 10 Ave 40.749717753 -74.002950346 459.0 W 20 St & 11 Ave 40.746745 -74.007756 19816.0 Customer nan 0.0 15887.0
2 178.0 1.372662064e+12 1.372662242e+12 293.0 Lafayette St & E 8 St 40.73028666 -73.9907647 237.0 E 11 St & 2 Ave 40.73047309 -73.98672378 14548.0 Subscriber 1980.0 2.0 15887.0
3 1580.0 1.372662066e+12 1.372663646e+12 531.0 Forsyth St & Broome St 40.71893904 -73.99266288 499.0 Broadway & W 60 St 40.76915505 -73.98191841 16063.0 Customer nan 0.0 15887.0
4 757.0 1.37266207e+12 1.372662827e+12 382.0 University Pl & E 14 St 40.73492695 -73.99200509 410.0 Suffolk St & Stanton St 40.72066442 -73.98517977 19213.0 Subscriber 1986.0 1.0 15887.0
5 861.0 1.372662083e+12 1.372662944e+12 511.0 E 14 St & Avenue B 40.72938685 -73.97772429 454.0 E 51 St & 1 Ave 40.75455731 -73.96592976 16223.0 Subscriber 1988.0 1.0 15887.0
6 550.0 1.372662119e+12 1.372662669e+12 293.0 Lafayette St & E 8 St 40.73028666 -73.9907647 394.0 E 9 St & Avenue C 40.72521311 -73.97768752 16746.0 Customer nan 0.0 15887.0
7 288.0 1.372662136e+12 1.372662424e+12 224.0 Spruce St & Nassau St 40.71146364 -74.00552427 376.0 John St & William St 40.70862144 -74.00722156 16062.0 Subscriber 1985.0 2.0 15887.0
8 766.0 1.372662136e+12 1.372662902e+12 432.0 E 7 St & Avenue A 40.72621788 -73.98379855 336.0 Sullivan St & Washington Sq 40.73047747 -73.99906065 17963.0 Subscriber 1980.0 2.0 15887.0
9 773.0 1.372662143e+12 1.372662916e+12 173.0 Broadway & W 49 St 40.76064679 -73.98442659 479.0 9 Ave & W 45 St 40.76019252 -73.9912551 19365.0 Subscriber 1989.0 1.0 15887.0

In [7]:
# Now do a monster Group-By.  Count bike starts per-station per-day.  Ends up
# with about 340 stations times 400 days (140,000 rows).  This is what we want
# to predict.
group_by_cols = ["Days","start station name"]
aggregates = {"bikes": ["count", 0, "all"]}
bpd = data.group_by(cols=group_by_cols, aggregates=aggregates) # Compute bikes-per-day
bpd.show()
bpd.describe()
bpd.dim()

TypeError: group_by() got an unexpected keyword argument 'cols'

In [ ]:
# Quantiles: the data is fairly unbalanced; some station/day combos are wildly
# more popular than others.
print "Quantiles of bikes-per-day"
bpd["bikes"].quantile().show()

In [ ]:
# A little feature engineering
# Add in month-of-year (seasonality; fewer bike rides in winter than summer)
secs = bpd["Days"]*secsPerDay
bpd["Month"]     = secs.month().asfactor()
# Add in day-of-week (work-week; more bike rides on Sunday than Monday)
bpd["DayOfWeek"] = secs.dayOfWeek()
print "Bikes-Per-Day"
bpd.describe()

In [ ]:
# ----------
# 3- Fit a model on train; using test as validation

# Function for doing class test/train/holdout split
def split_fit_predict(data):
  global gbm0,drf0,glm0,dl0
  # Classic Test/Train split
  r = data['Days'].runif()   # Random UNIForm numbers, one per row
  train = data[  r  < 0.6]
  test  = data[(0.6 <= r) & (r < 0.9)]
  hold  = data[ 0.9 <= r ]
  print "Training data has",train.ncol(),"columns and",train.nrow(),"rows, test has",test.nrow(),"rows, holdout has",hold.nrow()
  
  # Run GBM
  s = time.time()
  gbm0 = h2o.gbm(x           =train.drop("bikes"),
                 y           =train     ["bikes"],
                 validation_x=test .drop("bikes"),
                 validation_y=test      ["bikes"],
                 ntrees=500, # 500 works well
                 max_depth=6,
                 learn_rate=0.1)
  gbm_elapsed = time.time() - s

  # Run DRF
  s = time.time()
  drf0 = h2o.random_forest(x =train.drop("bikes"),
                y           =train     ["bikes"],
                validation_x=test .drop("bikes"),
                validation_y=test      ["bikes"],
                ntrees=250,
                max_depth=30)
  drf_elapsed = time.time() - s 
    
    
  # Run GLM
  s = time.time()
  glm0 = h2o.glm(x           =train.drop("bikes"),
                 y           =train     ["bikes"],
                 validation_x=test .drop("bikes"),
                 validation_y=test      ["bikes"],
                 Lambda=[1e-5],
                 family="poisson")
  glm_elapsed = time.time() - s
  
  # Run DL
  s = time.time()
  dl0 = h2o.deeplearning(x  =train.drop("bikes"),
                y           =train     ["bikes"],
                validation_x=test .drop("bikes"),
                validation_y=test      ["bikes"],
                hidden=[50,50,50,50],
                epochs=50)
  dl_elapsed = time.time() - s
  
  # ----------
  # 4- Score on holdout set & report
  train_r2_gbm = gbm0.model_performance(train).r2()
  test_r2_gbm  = gbm0.model_performance(test ).r2()
  hold_r2_gbm  = gbm0.model_performance(hold ).r2()
#   print "GBM R2 TRAIN=",train_r2_gbm,", R2 TEST=",test_r2_gbm,", R2 HOLDOUT=",hold_r2_gbm
  
  train_r2_drf = drf0.model_performance(train).r2()
  test_r2_drf  = drf0.model_performance(test ).r2()
  hold_r2_drf  = drf0.model_performance(hold ).r2()
#   print "DRF R2 TRAIN=",train_r2_drf,", R2 TEST=",test_r2_drf,", R2 HOLDOUT=",hold_r2_drf
  
  train_r2_glm = glm0.model_performance(train).r2()
  test_r2_glm  = glm0.model_performance(test ).r2()
  hold_r2_glm  = glm0.model_performance(hold ).r2()
#   print "GLM R2 TRAIN=",train_r2_glm,", R2 TEST=",test_r2_glm,", R2 HOLDOUT=",hold_r2_glm
    
  train_r2_dl = dl0.model_performance(train).r2()
  test_r2_dl  = dl0.model_performance(test ).r2()
  hold_r2_dl  = dl0.model_performance(hold ).r2()
#   print " DL R2 TRAIN=",train_r2_dl,", R2 TEST=",test_r2_dl,", R2 HOLDOUT=",hold_r2_dl
    
  # make a pretty HTML table printout of the results

  header = ["Model", "R2 TRAIN", "R2 TEST", "R2 HOLDOUT", "Model Training Time (s)"]
  table  = [
            ["GBM", train_r2_gbm, test_r2_gbm, hold_r2_gbm, round(gbm_elapsed,3)],
            ["DRF", train_r2_drf, test_r2_drf, hold_r2_drf, round(drf_elapsed,3)],
            ["GLM", train_r2_glm, test_r2_glm, hold_r2_glm, round(glm_elapsed,3)],
            ["DL ", train_r2_dl,  test_r2_dl,  hold_r2_dl , round(dl_elapsed,3) ],
           ]
  h2o.H2ODisplay(table,header)
  # --------------

In [ ]:
# Split the data (into test & train), fit some models and predict on the holdout data
split_fit_predict(bpd)
# Here we see an r^2 of 0.91 for GBM, and 0.71 for GLM.  This means given just
# the station, the month, and the day-of-week we can predict 90% of the
# variance of the bike-trip-starts.

In [ ]:
# ----------
# 5- Now lets add some weather
# Load weather data
wthr1 = h2o.import_frame(path=[mylocate("bigdata/laptop/citibike-nyc/31081_New_York_City__Hourly_2013.csv"),
                               mylocate("bigdata/laptop/citibike-nyc/31081_New_York_City__Hourly_2014.csv")])
# Peek at the data
wthr1.describe()

In [ ]:
# Lots of columns in there!  Lets plan on converting to time-since-epoch to do
# a 'join' with the bike data, plus gather weather info that might affect
# cyclists - rain, snow, temperature.  Alas, drop the "snow" column since it's
# all NA's.  Also add in dew point and humidity just in case.  Slice out just
# the columns of interest and drop the rest.
wthr2 = wthr1[["Year Local","Month Local","Day Local","Hour Local","Dew Point (C)","Humidity Fraction","Precipitation One Hour (mm)","Temperature (C)","Weather Code 1/ Description"]]

wthr2.setName(wthr2.index("Precipitation One Hour (mm)"), "Rain (mm)")
wthr2.setName(wthr2.index("Weather Code 1/ Description"), "WC1")
wthr2.describe()
# Much better!  

In [ ]:
# Filter down to the weather at Noon
wthr3 = wthr2[ wthr2["Hour Local"]==12 ]

In [ ]:
# Lets now get Days since the epoch... we'll convert year/month/day into Epoch
# time, and then back to Epoch days.  Need zero-based month and days, but have
# 1-based.
wthr3["msec"] = h2o.H2OFrame.mktime(year=wthr3["Year Local"], month=wthr3["Month Local"]-1, day=wthr3["Day Local"]-1, hour=wthr3["Hour Local"])
secsPerDay=1000*60*60*24
wthr3["Days"] = (wthr3["msec"]/secsPerDay).floor()
wthr3.describe()
# msec looks sane (numbers like 1.3e12 are in the correct range for msec since
# 1970).  Epoch Days matches closely with the epoch day numbers from the
# CitiBike dataset.  

In [ ]:
# Lets drop off the extra time columns to make a easy-to-handle dataset.
wthr4 = wthr3.drop("Year Local").drop("Month Local").drop("Day Local").drop("Hour Local").drop("msec")

In [ ]:
# Also, most rain numbers are missing - lets assume those are zero rain days
rain = wthr4["Rain (mm)"]
rain[ rain.isna() ] = 0

In [ ]:
# ----------
# 6 - Join the weather data-per-day to the bike-starts-per-day
print "Merge Daily Weather with Bikes-Per-Day"
bpd_with_weather = bpd.merge(wthr4,allLeft=True,allRite=False)
bpd_with_weather.describe()
bpd_with_weather.show()

In [ ]:
# 7 - Test/Train split again, model build again, this time with weather
split_fit_predict(bpd_with_weather)